# Microsoft Graph

“[Build .NET apps with Microsoft Graph and app-only authentication](https://learn.microsoft.com/en-us/graph/tutorials/dotnet-app-only?tabs=aad)”

In [1]:
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.Core,1.31"
#r "nuget: Azure.Identity"

Installed Packages Azure.Core, 1.31.0 Azure.Identity, 1.8.2 Microsoft.Graph, 5.9.0

In [2]:
#r "nuget: SonghayCore"

Installed Packages SonghayCore, 6.0.4

In [3]:
using System.IO;
using System.Text.Json;
using Songhay.Models;

var json = File.ReadAllText(Environment.GetEnvironmentVariable("SONGHAY_APP_SETTINGS_PATH"));

JsonDocument.Parse(json).RootElement.TryGetProperty(typeof(ProgramMetadata).Name, out var element)

True

In [4]:
var meta = JsonSerializer.Deserialize<ProgramMetadata>(element);
var restMeta = meta.RestApiMetadataSet["MSAL2023"];

restMeta.ClaimsSet.Keys

[ ClientId, ClientScope, ClientSecret, ClientTenantId ]

In [19]:
using Azure.Core;
using Azure.Identity;

var credential = new ClientSecretCredential(
    restMeta.ClaimsSet["ClientTenantId"],
    restMeta.ClaimsSet["ClientId"],
    restMeta.ClaimsSet["ClientSecret"]
);

var scopes = new[] { "api://9b0e1cae-2015-46ba-9be9-ddbacddcd316/.default" };

var context = new TokenRequestContext(scopes);

var response = await credential.GetTokenAsync(context);

!string.IsNullOrWhiteSpace(response.Token)

True

In [6]:
#r "nuget: Azure.Storage.Blobs"


Installed Packages Azure.Storage.Blobs, 12.16.0

In [20]:
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

var uri = new Uri("https://fubar.blob.core.windows.net", UriKind.Absolute);
var client = new BlobServiceClient(uri, credential);

var containerClient = client.GetBlobContainerClient("foo");

var blobClient = containerClient.GetBlobClient("kb-1064618963.json");

blobClient.DownloadContent()

Error: Azure.RequestFailedException: This request is not authorized to perform this operation using this permission.
RequestId:f3a7242e-601e-0019-4ec5-80a3c2000000
Time:2023-05-07T09:25:34.2932760Z
Status: 403 (This request is not authorized to perform this operation using this permission.)
ErrorCode: AuthorizationPermissionMismatch

Headers:
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-request-id: f3a7242e-601e-0019-4ec5-80a3c2000000
x-ms-client-request-id: d9f1d359-127c-4f8c-b8ba-af5797454684
x-ms-version: 2022-11-02
x-ms-error-code: AuthorizationPermissionMismatch
Access-Control-Allow-Origin: *
Date: Sun, 07 May 2023 09:25:33 GMT
Content-Length: 279
Content-Type: application/xml

   at Azure.Storage.Blobs.BlobRestClient.Download(String snapshot, String versionId, Nullable`1 timeout, String range, String leaseId, Nullable`1 rangeGetContentMD5, Nullable`1 rangeGetContentCRC64, String encryptionKey, String encryptionKeySha256, Nullable`1 encryptionAlgorithm, Nullable`1 ifM